In [1]:
from profilehooks import profile
import pandas as pd
import datetime
from pull_datacb import getCBHistory

In [2]:
class MaxMin:

    def __init__(self, df: pd.DataFrame, period: int):
        self.df     = df
        self.period = period

    def update_cert(self, cert, index, df):
        #TODO: Implement ittertool module
        # Create certifying candle
        # count number of null values to see if is first row
        null_count = 0
        for value in cert.values():
            if value == None:
                null_count += 1
        
        # update prev dictionary if all null values
        # if first row it will update with those values
        if null_count == 5:
            cert['t'] = index
            cert['o'] = float(df.loc[index]['Open'])
            cert['c'] = float(df.loc[index]['Close'])
            cert['h'] = float(df.loc[index]['High'])
            cert['l'] = float(df.loc[index]['Low'])
            
            return cert
        
        # update prev with new prev values (new prev is old curr)
        else:
            if index + datetime.timedelta(seconds=self.period):
                cert['t'] = (index)
                cert['o'] = float(df.loc[index]['Open'])
                cert['c'] = float(df.loc[index]['Close'])
                cert['h'] = float(df.loc[index]['High'])
                cert['l'] = float(df.loc[index]['Low'])
            
                return cert


    def update_curr(self, curr, index, df):
        # format datetime index
        dt_object = (index - datetime.timedelta(seconds=self.period)).to_datetime64()
        
        if (df.index == dt_object).any():
            index -= datetime.timedelta(seconds=self.period)
            curr['t'] = (index)
            curr['o'] = float(df.loc[index]['Open'])
            curr['c'] = float(df.loc[index]['Close'])
            curr['h'] = float(df.loc[index]['High'])
            curr['l'] = float(df.loc[index]['Low'])
            
            # return the current candle dictionary
            return curr
        else:
            pass


    def update_prev(self, prev, index, df):
        dt_object = (index - datetime.timedelta(seconds=2*self.period)).to_datetime64()

        if (df.index == dt_object).any():
            index -= datetime.timedelta(seconds=2*self.period)
            prev['t'] = (index)
            prev['o'] = float(df.loc[index]['Open'])
            prev['c'] = float(df.loc[index]['Close'])
            prev['h'] = float(df.loc[index]['High'])
            prev['l'] = float(df.loc[index]['Low'])
            
            # return the previous candle dictionary
            return prev
        else:
            pass


    def find_max(self, cert_candle, curr_candle, prev_candle):
        # get local max
        max_dict = {}
        if curr_candle and prev_candle:       
            if (curr_candle['c'] > cert_candle['c']) & (curr_candle['c'] > prev_candle['c']):
                max_dict = curr_candle
                return max_dict
            else:
                return None


    def find_min(self, cert_candle, curr_candle, prev_candle):   
        # get local min
        min_dict = {}
        if curr_candle and prev_candle:
            if (curr_candle['c'] < cert_candle['c']) & (curr_candle['c'] < prev_candle['c']):
                min_dict = curr_candle
                return min_dict
            else:
                return None


    def create_dataframe(self, df):
        """
        Create DataFrame with local minimums and maximums to create signals for double bottom creation
        Accepts DataFrames with columns formatted as 'Open', 'Close', 'High', 'Low'.
        More columns can be added.
        """
        
        # # store previous candle
        prev = {
            't' : None,
            'o' : None,
            'c' : None,
            'h' : None,
            'l' : None   
        }

        # store current candle
        curr = {
            't' : None,
            'o' : None,
            'c' : None,
            'h' : None,
            'l' : None 
        }

        # store certifying candle (certifies if the max/min is created)
        cert = {
            't' : None,
            'o' : None,
            'c' : None,
            'h' : None,
            'l' : None 
        }

        # Init lists for max and min events
        max_ = []
        min_ = []

        # Iterrate through dataframes indexes
        for index, row in df.iterrows():
            cert_candle = self.update_cert(cert, index, df)
            curr_candle = self.update_curr(curr, index, df)
            prev_candle = self.update_prev(prev, index, df)
            
            mx = self.find_max(cert_candle, curr_candle, prev_candle)
            mn = self.find_min(cert_candle, curr_candle, prev_candle)

            # max stores close and high
            # min stores close and low
            if mx == None:
                max_.append(0)
            else:
                max_.append(1)

            if mn == None:
                min_.append(0)      
            else:
                min_.append(1)

        # Adding min/max events to original dataframe then creating a copy with columns for double bottom events
        max_min_df = pd.DataFrame({'Max': max_, 'Min': min_})
        max_min_df = max_min_df[['Max', 'Min']].shift(-1)
        max_min_df['Date'] = df.index
        max_min_df.set_index('Date', inplace=True)
        df_features = pd.concat([df, max_min_df], axis=1)
        
        return df_features

    
    def output(self):
        return self.create_dataframe(self.df)

In [6]:
min        = 60
five_min   = 300
fiften_min = 900
hour       = 3600
four_hour  = 21600
day        = 86400

def get_maxmin(symbol, period, cycles):
    # Gets df of local maximums and minimums
    df      = getCBHistory(symbol, period, cycles=cycles)
    max_min = MaxMin(df, period)
    return max_min.output()


dataframe = get_maxmin('BTC', min, 4)

In [32]:
def slope(dataframe):
    # Get slope of last two minimum values
    minimums = dataframe.loc[dataframe['Min'] == 1]
    last_four = minimums.tail(2)
    last_four.reset_index(inplace=True)

    first_point  = []
    second_point = []
    for i, row in last_four.iterrows():
        if i == 0:
            dt = last_four.iloc[i]['Date']
            first_point.append(dt)
            price = last_four.iloc[i]['Close']
            first_point.append(price)
        if i == 1:
            dt = last_four.iloc[i]['Date']
            second_point.append(dt)
            price = last_four.iloc[i]['Close']
            second_point.append(price)

    change_y = second_point[1] - first_point[1]
    change_x = (second_point[0] - first_point[0]).total_seconds()

    return change_y/change_x

slope = slope(
    dataframe=dataframe
)

display(slope)

-0.019194444444454145

In [59]:
df = getCBHistory('BTC', 60, cycles=1)
for index, rows in df.iterrows():
    print(type(index))

<class 'pandas._libs.tslibs.timestamps.Timestamp'>
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
<class 'pandas._libs.tslibs.tim

In [73]:
df.index[-1]

Timestamp('2022-01-26 14:49:00')